In [ ]:
import pandas as pd
import numpy as np
import polars as pl
import seaborn as sns
import matplotlib.pyplot as plt
import json
from matplotlib.colors import Normalize
import tableone

with open('../params.json', 'r') as file :
    params = json.load(file)

DATASET, VERSION = params['dataset'], params['version']
DATA_FOLD = params['data_folder']


# Import Datasets

In [ ]:
data = pl.read_parquet(f'{DATA_FOLD}/{VERSION}/3.analysis/imputation_48/{DATASET}/first_48h.parquet')

# Patients descriptions

In [ ]:
id_list = data['encounterId'].unique().to_list()

In [ ]:
demo = pl.read_parquet(f"{DATA_FOLD}/{VERSION}/2.clean_data/{DATASET}/static/clean_static_encounters.parquet").filter(pl.col('encounterId').is_in(id_list))

In [ ]:
demo = (demo
        .with_columns(
            year_inTime = pl.col('utcInTime').dt.year(),
            month_outTime = pl.col('utcOutTime').dt.month(),
            bmi = (pl.col('poids_admission') / (pl.col('taille') / 100) ** 2).round(1),
            los_days = (pl.col('los')/24).round(1)
            )
        )

In [ ]:
var_demo = ['age', 'gender', 'bmi', 'los_days', 'admission_type', 'sapsii', 'isDeceased']
categorical = [ 'gender','admission_type', 'isDeceased']
demo_pandas = demo.to_pandas()

In [ ]:
mytable = tableone.TableOne(demo_pandas, var_demo, categorical, pval=False, missing=False)
print(mytable.tabulate(tablefmt="latex"))

# Time Series Data

In [ ]:
data_features = ['heart_rate', 'spo2', 'fr', 'pam']

In [ ]:
data_df = data[data_features]

## Heart rate

In [ ]:
sns.histplot(data, x='heart_rate', bins=100, stat="density")
plt.xlabel('Heart Rate')
plt.ylabel('')

## MAP

In [ ]:
sns.histplot(data, x='pam', bins=100, stat="density")
plt.xlabel('Mean Arterial Pressure')
plt.ylabel('')

# SpO2

In [ ]:
sns.histplot(data, x='spo2', bins=100, stat="density")
plt.xlabel('SpO2')
plt.ylabel('')

# Respiratory rate

In [ ]:
sns.histplot(data, x='fr', bins=100, stat="density")
plt.xlabel('Respiratory Rate')
plt.ylabel('')

In [ ]:
data['heart_rate',
 'spo2',
 'fr',
 'pad',
 'pam',
 'pas',].to_pandas().corr()